# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that the Retail Demo Store has trained models for 3 different use cases (Recommended for You, Customers who viewed X also viewed and Ranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [41]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [42]:
#retrieves previously stored variables 
%store -r 

In [43]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Adding some helper functions to make results more readable

First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`, and potentially information about whether that item was part of a promotion filter. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the synthetic dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: If you want to lookup information as part of the API request, consider implementing Real-Time Personalization APIs - https://github.com/aws-samples/personalization-apis.
</div>



In [44]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [45]:
def get_item_category_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['category'].values[0]
    return item_name

In [46]:
def get_item_style_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['style'].values[0]
    return item_name

Great! Now we have a way of rendering results. 

## Interact with Recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "Customers who viewed X also viewed" Recommender

"Customers who viewed X also viewed" requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, `numResults`.

#### Select a User

We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [47]:
user_id = 555
user_metadata_df[user_metadata_df['id']==user_id]

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
554,555,True,F,Natalie,Smith,natalie.smith@example.com,51,Natalie Smith,user555,outdoors_instruments_groceries,lower_priced_products,{},{'ios': {'anonymous_id': '6860933c-dce7-4ef0-b...,"[{'first_name': 'Natalie', 'last_name': 'Smith..."


We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [48]:
product_id = item_metadata_df.sample(1)["id"].values[0]

print ('productId: ', product_id)

productId:  e80950d7-b0f8-473d-a06f-c5ec49219001


First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [49]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_customers_who_viewed_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 20
)

In [50]:
print (get_recommendations_response)

{'ResponseMetadata': {'RequestId': 'a853e077-f40e-4b5f-87bf-b7b983c4d659', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 22 Aug 2024 17:21:52 GMT', 'content-type': 'application/json', 'content-length': '2736', 'connection': 'keep-alive', 'x-amzn-requestid': 'a853e077-f40e-4b5f-87bf-b7b983c4d659'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '4f6f36db-0808-4ead-80c7-82f2fa08b5cb'}, {'itemId': '5dcabfe7-c137-4884-a78f-15bd313ec491'}, {'itemId': 'bd1f78d6-393b-4269-9c5b-300700bb9dab'}, {'itemId': 'f1e0660b-53db-4e9a-a86a-8a990d6b2988'}, {'itemId': 'ae44cb9e-05e7-4802-aa80-bc995c87907a'}, {'itemId': '6377bbe2-0972-4376-a797-1af84af9a3a6'}, {'itemId': 'dfb69542-c032-4c9a-bc06-ddd6282fbe37'}, {'itemId': 'ecfc70c9-a9a3-427e-91ea-5dd561a2564a'}, {'itemId': 'd16257f7-da83-4b24-9e2f-5c4a2a67bdc9'}, {'itemId': '2314f923-cffd-4286-aaf1-fd016bf8a06d'}, {'itemId': 'e91ea7d3-17d4-46a4-bf60-1e675351d828'}, {'itemId': '65b3bc05-4d84-48ef-ba0f-ce12301643c7'}, {'itemId': '0d3c54a9-314d-453a-913a-

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for products render in a nice dataframe. Again, let's create a helper function to achieve this.

In [51]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_name_from_id(item_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_customers_who_viewed_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [52]:
samples = item_metadata_df.sample(5)
samples

,id,name,category,style,featured
356,880ac468-d596-4332-9920-e5f96176ef65,Cozy Chic Knit-Collar Coat,apparel,scarf,NaN
2453,7a619c82-a5da-4bc9-b6e6-64e93c51fb55,Gourmet Roasted Cashews Delight,salty snacks,nuts/seeds,NaN
2153,11d3acc6-c567-45a2-a865-15723b41b162,Festive Santa Hat Cocktail Picks,seasonal,christmas,NaN
941,2d2d8ec8-4806-42a7-b8ba-ceb15c1c7e84,Retro Stripe Urban Sneakers,footwear,sneaker,NaN
1714,1f7340f0-805d-4ba9-a0e4-76d392eab5a1,Damascus Steel Chef's Knife,housewares,kitchen,NaN


In [53]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.id.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df

,Cozy Chic Knit-Collar Coat,Gourmet Roasted Cashews Delight,Festive Santa Hat Cocktail Picks,Retro Stripe Urban Sneakers,Damascus Steel Chef's Knife
0,Classic Buffalo Plaid Flannel Shirt,Succulent Grilled Jumbo Shrimp Platter,Romantic Marble Heart Placemat,Navy Canvas Urban Sneaker,Citrus Splash Nonstick Wok
1,Emerald Velvet Cushion Covers,Fiesta Nacho Platter,Luxe Artisan Chocolate Assortment,Vintage Canvas Wingtip Oxfords,Moroccan Handcrafted Ceramic Bowl Set
2,Classic Checkered Flannel Shirt,Classic Pepperoni Pizza Slice,Spooky Treats Halloween Mug Set,Urban Splash Runners,Sunset Glow Dutch Oven
3,Cozy Cabin Wool Socks,Gourmet Raw Spanish Peanuts,Halloween Ambiance Candle Set,Stealth Shadow Runner,Spicy Singaporean Laksa Bowl
4,Elegant Timepiece Chronograph,Natural Almond Delight,Vibrant Mexican Calavera Skull,Sunburst Mid-Century Accent Chair,Ruby Twilight Craft Beer
5,Vintage Burgundy Leather Jacket,Refreshing Cola Delight,Cuddly Angel Bear Plush,Minimalist Metal Desk,Rustic Elegance Table Setting Kit
6,Rugged Wilderness Parka,Tropical Smoothie Party Pack,Rustic Smart Home Office Desk,Rugged Explorer Leather Boots,Classic Red Ceramic Teapot Set
7,Mountain Peak Adventure Socks,Sunset Sips Whiskey Glasses Set,Premium Seasoned Beef Strips,Cozy Winter Wonderland Socks,Crystal Clarity Herbal Teapot
8,Coral Breeze Beach Slides,NaN,Modern Lounge Oasis Set,Crimson Tide High-Top Sneaker,Savernake Ultimate Knife Set
9,Vintage Charm Oil Lantern,NaN,Rustic Jingle Bell Wreath Ornament,Rugged Terrain Hiking Boots,Elegance Stemless Whiskey Glass Set


Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the "Customers who viewed X also viewed" recommendations are based on the interactions we generated as input into the solution creation process above.

You may notice that some items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with a small dataset).

### "Recommended for you" Recommender

"Recommended for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

With this use case, Amazon Personalize automatically filters items the user purchased based on the `userId` that you specify and "Purchase" events. For better performance, include "Purchase" events along with the required "View" events. 

In [54]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20
)

In [55]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)


,Item,Name,Category,Style
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,Minimalist Circular Wall Mirror,instruments,percussion
1,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,Elegant Grand Piano Suite,instruments,keys
2,b4e0ec0f-efec-4103-a741-d9e373808130,Vintage Cream Electric Guitar,instruments,strings
3,9f251479-7904-4563-b826-cd4f3739949b,SunsetGlow Expedition Tent,outdoors,camping
4,e4f7e8b7-2c26-4939-9efd-b10f53a2ed04,Portable MIDI Keyboard Controller,instruments,keys
5,3f91dc0a-b158-47d4-a6ad-9bbf5c6560e5,Melodic Woodwind Ensemble,instruments,wind
6,55f532d2-3067-4b48-9186-aba676681dfc,Pro Angler's Delight Fishing Rod,outdoors,fishing
7,94610697-2580-432a-8479-ee19686df56c,Shamanic Drum Experience Kit,instruments,percussion
8,e2a60c0c-de0c-49d0-b66f-cf6406058797,Premium Diced Beef Chunks,groceries,meat
9,849bb1b3-da1b-451b-8151-879a18e11437,Vintage Glow Lantern Lamp,outdoors,camping


Since "Recommended for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [56]:
samples = user_metadata_df.sample(3)
samples

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
647,648,True,F,Michelle,Chang,michelle.chang@example.com,41,Michelle Chang,user648,electronics_outdoors_footwear,lower_priced_products,{},{'ios': {'anonymous_id': '7ba1df41-0b7f-4b20-a...,"[{'first_name': 'Michelle', 'last_name': 'Chan..."
4620,4621,True,M,Marvin,Wallace,marvin.wallace@example.com,49,Marvin Wallace,user4621,accessories_groceries_books,discount_indifferent,{},{'ios': {'anonymous_id': '99626fb0-fee2-4e20-9...,"[{'first_name': 'Marvin', 'last_name': 'Wallac..."
2760,2761,True,F,Victoria,Rojas,victoria.rojas@example.com,23,Victoria Rojas,user2761,homedecor_electronics_outdoors,all_discounts,{},{'ios': {'anonymous_id': '87d91939-bc80-43bf-b...,"[{'first_name': 'Victoria', 'last_name': 'Roja..."


Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Recommended for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a product.

The cells below will handle getting recommendations from the "Recommended for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [57]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_recommended_for_you(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [58]:
recommendations_df_users = pd.DataFrame()
users = samples.id.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_recommended_for_you(recommendations_df_users, user)

recommendations_df_users

,648,4621,2761
0,Quirky Milk Bottle Night Light,Urban Graffiti Question Backpack,Modern Halo Pendant Light
1,Sunkissed Braided Thong Sandals,Vintage Chic Clubmaster Glasses,Sleek Brass Adjustable Desk Lamp
2,Chic Noir Wedge Sandals,Sleek Sport Optical Frames,Scandinavian Tripod Floor Lamp
3,Blush Elegance Strappy Sandals,Classic Traveler's Timepiece,Geometric Chic Throw Pillow Set
4,Elegant Blossom Bridal Pumps,Sunshine Bunch Organic Bananas,Rustic Charm Floral Basket Set
5,Regal Ruby Velvet Flats,Artisan Two-Tone Sesame Bagel,Moroccan Kilim Pillow Set
6,Glittering Glamour Bridal Heels,Garden-Fresh Gourmet Cucumbers,Sleek Adjustable Wall Sconce
7,Sleek Noir Wireless Headphones,Homestyle Butter Cookies,Rustic Bamboo Cage Pendant Light
8,Premium Woven Audio Masterpiece,Urban Stealth Backpack,Copper Dome Table Lamp
9,Starlit Nights Peep-Toe Booties,Adventure Seeker Hiking Backpack,Scandinavian Ombre Pendant Light


## Applying the discount context

We'll get the user recommendations when discount context is applied for comparison. This is using the "contextual metadata" feature of Amazon Personalize. In this case, our context is whether a discount was applied.

In [59]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20,
    context = {'DISCOUNT':'Yes'} # Here we provide the context for the recommendations
)

item_list_context = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list_context ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list_context ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item,Name,Category,Style
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,Minimalist Circular Wall Mirror,instruments,percussion
1,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,Elegant Grand Piano Suite,instruments,keys
2,b4e0ec0f-efec-4103-a741-d9e373808130,Vintage Cream Electric Guitar,instruments,strings
3,9f251479-7904-4563-b826-cd4f3739949b,SunsetGlow Expedition Tent,outdoors,camping
4,3f91dc0a-b158-47d4-a6ad-9bbf5c6560e5,Melodic Woodwind Ensemble,instruments,wind
5,e4f7e8b7-2c26-4939-9efd-b10f53a2ed04,Portable MIDI Keyboard Controller,instruments,keys
6,55f532d2-3067-4b48-9186-aba676681dfc,Pro Angler's Delight Fishing Rod,outdoors,fishing
7,94610697-2580-432a-8479-ee19686df56c,Shamanic Drum Experience Kit,instruments,percussion
8,849bb1b3-da1b-451b-8151-879a18e11437,Vintage Glow Lantern Lamp,outdoors,camping
9,d792631e-1d04-4995-a5be-ea7d56ea1882,LakesideEscape Wilderness Tent Set,outdoors,camping


Let us compare it to the previous set of recommendations without context.

In [60]:
df = pd.DataFrame()
df['Item (No context)'] = [ itm['itemId'] for itm in item_list ]
df['Name (No context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]

df['Item (Context)'] = [ itm['itemId'] for itm in item_list_context ]
df['Name (Context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item (No context),Name (No context),Item (Context),Name (Context)
0,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,Minimalist Circular Wall Mirror,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,Minimalist Circular Wall Mirror
1,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,Elegant Grand Piano Suite,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,Elegant Grand Piano Suite
2,b4e0ec0f-efec-4103-a741-d9e373808130,Vintage Cream Electric Guitar,b4e0ec0f-efec-4103-a741-d9e373808130,Vintage Cream Electric Guitar
3,9f251479-7904-4563-b826-cd4f3739949b,SunsetGlow Expedition Tent,9f251479-7904-4563-b826-cd4f3739949b,SunsetGlow Expedition Tent
4,e4f7e8b7-2c26-4939-9efd-b10f53a2ed04,Portable MIDI Keyboard Controller,3f91dc0a-b158-47d4-a6ad-9bbf5c6560e5,Melodic Woodwind Ensemble
5,3f91dc0a-b158-47d4-a6ad-9bbf5c6560e5,Melodic Woodwind Ensemble,e4f7e8b7-2c26-4939-9efd-b10f53a2ed04,Portable MIDI Keyboard Controller
6,55f532d2-3067-4b48-9186-aba676681dfc,Pro Angler's Delight Fishing Rod,55f532d2-3067-4b48-9186-aba676681dfc,Pro Angler's Delight Fishing Rod
7,94610697-2580-432a-8479-ee19686df56c,Shamanic Drum Experience Kit,94610697-2580-432a-8479-ee19686df56c,Shamanic Drum Experience Kit
8,e2a60c0c-de0c-49d0-b66f-cf6406058797,Premium Diced Beef Chunks,849bb1b3-da1b-451b-8151-879a18e11437,Vintage Glow Lantern Lamp
9,849bb1b3-da1b-451b-8151-879a18e11437,Vintage Glow Lantern Lamp,d792631e-1d04-4995-a5be-ea7d56ea1882,LakesideEscape Wilderness Tent Set


## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For an ECOMMERCE application you want to dynamically render a personalized shelf/rail/carousel based on some information (category, style, season, etc…). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within your system to generate the item list. You can use this campaign to rank the products listed for each category and the featured products list, as well as ranking catalog search results displayed in a search widget.

To demonstrate this, we will use the same user from before and a random collection of items.

#### Get Featured Products List

First let's get the list of featured products from the Products data.

In [61]:
featured_products = item_metadata_df[item_metadata_df['featured']==True]

#### Rank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rank for a specific user. This ranking allows us to provide ranked products based on the user's behavior. These behaviors should be consistent to the same persona that was mentioned above (since we're going to use the same `user_id`).

In [62]:
unranked_product_ids = list (featured_products['id'])

df = pd.DataFrame()
df['Item'] = [ itm for itm in unranked_product_ids ]
df['Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]
df['Category'] = [ get_item_category_from_id ( itm) for itm in unranked_product_ids ]
df['Style'] = [ get_item_style_from_id ( itm) for itm in unranked_product_ids ]
display (df)

,Item,Name,Category,Style
0,2b67230f-dc22-462e-9afe-c9e459f74093,Sunburst Rattan Crossbody Bag,accessories,handbag
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Vibrant Clash Red Blazer Set,apparel,jacket
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Lavender Bliss Artisan Soap Set,beauty,bathing
3,5d37a44b-d121-426e-b528-59e603ba5923,Egyptian Pyramid Adventure Package,books,travel
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,Sleek Noir Wireless Headphones,electronics,headphones
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Ethereal Blush Bouquet,floral,bouquet
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Urban Glide Red Sneaker Wedges,footwear,sneaker
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Rustic Charm Sofa Set,furniture,sofas
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Fresh-Cut Exotic Kiwi Duo,groceries,fruits
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Cheerful Succulent Planter Pot,homedecor,decorative


Now let's have Personalize rank the featured product IDs based on our random user.

In [63]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

[
    {
        "itemId": "7160b264-e3ed-4ac3-9dd7-2c537b00e5ed",
        "score": 0.7982783
    },
    {
        "itemId": "3f9a39b2-0d63-4751-b6ee-4ecd08dd2276",
        "score": 0.1589923
    },
    {
        "itemId": "a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec",
        "score": 0.0248708
    },
    {
        "itemId": "5d37a44b-d121-426e-b528-59e603ba5923",
        "score": 0.0077695
    },
    {
        "itemId": "3b145528-d5fc-4c2a-b2a5-e119128caa5f",
        "score": 0.0059969
    },
    {
        "itemId": "22552eb1-57f1-4fa3-a93a-a9fa22851f9f",
        "score": 0.0019145
    },
    {
        "itemId": "2b67230f-dc22-462e-9afe-c9e459f74093",
        "score": 0.0009571
    },
    {
        "itemId": "8bffb5fb-624f-48a8-a99f-b8e9c64bbe29",
        "score": 0.0004392
    },
    {
        "itemId": "b87da3f8-9a3e-417d-abd7-16329c5be1ba",
        "score": 0.0002979
    },
    {
        "itemId": "6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b",
        "score": 0.0002219
    },
    {
        "ite

We will add the ranked items as a second column to the original dataframe, for a side-by-side comparison.

In [64]:
df = pd.DataFrame()
df['Original List'] = [ itm for itm in unranked_product_ids]
df['Original List Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]

df['Reranking'] = [ itm['itemId'] for itm in reranked]
df['Reranking Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in reranked ]

display (df)

,Original List,Original List Name,Reranking,Reranking Name
0,2b67230f-dc22-462e-9afe-c9e459f74093,Sunburst Rattan Crossbody Bag,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Active Pup Frisbee Set
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Vibrant Clash Red Blazer Set,3f9a39b2-0d63-4751-b6ee-4ecd08dd2276,Vintage Sunburst Electric Guitar
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Lavender Bliss Artisan Soap Set,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Fresh-Cut Exotic Kiwi Duo
3,5d37a44b-d121-426e-b528-59e603ba5923,Egyptian Pyramid Adventure Package,5d37a44b-d121-426e-b528-59e603ba5923,Egyptian Pyramid Adventure Package
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,Sleek Noir Wireless Headphones,3b145528-d5fc-4c2a-b2a5-e119128caa5f,Sleek Noir Wireless Headphones
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Ethereal Blush Bouquet,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Urban Glide Red Sneaker Wedges
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Urban Glide Red Sneaker Wedges,2b67230f-dc22-462e-9afe-c9e459f74093,Sunburst Rattan Crossbody Bag
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Rustic Charm Sofa Set,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Precision Pro Red Screwdriver
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Fresh-Cut Exotic Kiwi Duo,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Lavender Bliss Artisan Soap Set
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Cheerful Succulent Planter Pot,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Vibrant Clash Red Blazer Set


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "featured products" which are curated by a person.


# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be an accessories category filter, which invokes the get_recommendations_response api with the specific filter of CATEGORY_L1 = accessories. In order to create a recommendation for each filter, that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of CATEGORY_L1, where the actual category is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in ECOMMERCE are:

* Categorical filters based on Item Metadata - Often your item metadata will have information about the item. Filtering on these can provide recommendations within that data.

* Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on sale now".

* Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "recommended for you" recommendation to a "purchase again" recommendations.



### Create Category Filter

Since there are a lot of categories to filter on, we will create a dynamic filter using the dynamic variable $CATEGORY, this will allow us to pass in the variable at runtime rather than create a static filter for each category.

In [65]:
category_filter_name = 'filter-category'

try:
    create_category_filter_response = personalize.create_filter(
        name = category_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L1 IN ($CATEGORY)'
    )
    
    category_filter_arn = create_category_filter_response['filterArn']
    print('Creating the Personalize filter with category_filter_arn {}.'.format(category_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    category_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+category_filter_name
    print('The Personalize filter {} already exists.'.format(category_filter_name))
    print ('\nWe will be using the existing Personalize Filter with category_filter_arn = {}'.format(category_filter_arn))
    

The Personalize filter filter-category already exists.

We will be using the existing Personalize Filter with category_filter_arn = arn:aws:personalize:us-east-1:809697808660:filter/filter-category


### Create Purchased Products Filter

Since it's a poor user experience to recommend products that a user has already purchased, we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `Purchase` for the user. If you recall from the last notebook, the interactions dataset uses the `Purchase` event type to track purchased items.

In [66]:
filter_name = 'filter-purchased-products'

try:
    create_purchase_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("Purchase")'
    )
    
    purchase_filter_arn = create_purchase_filter_response['filterArn']

    print('Creating the Personalize filter with purchase_filter_arn {}.'.format(purchase_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    purchase_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with purchase_filter_arn = {}'.format(purchase_filter_arn))
    

The Personalize filter filter-purchased-products already exists.

We will be using the existing Personalize Filter with purchase_filter_arn = arn:aws:personalize:us-east-1:809697808660:filter/filter-purchased-products


### Wait for Filter Status to Become ACTIVE

The filter should take a minute or so to become active.

In [67]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = purchase_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Purchase filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)
    
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = category_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Category Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

Purchase filter: ACTIVE
Category Filter: ACTIVE


## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

## Category Filter<a class="anchor" id="filter">
[Back to top](#top)

To test our purchased products filter, we will request recommendations for a random user with a particular category. 
    
We will create a helper function to return recommendations using a dynamic flilter:

In [68]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, category_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
        filterArn = category_filter_arn,
        filterValues = { "CATEGORY": "\"" + filter_values + "\""},
        numResults = 10
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        product = get_item_name_from_id(item['itemId'])
        recommendation_list.append(product)
    filter_name = category_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will select the first 7 categories for this example.

In [29]:
unique_category_field_values = list(item_metadata_df['category'].unique()[:7])
unique_category_field_values

['accessories',
 'apparel',
 'beauty',
 'books',
 'electronics',
 'floral',
 'footwear']

In [30]:
# Iterate through Categories
recommendations_df_category = pd.DataFrame()
for category in unique_category_field_values:
    recommendations_df_category = get_new_recommendations_df_by_dynamic_filter(recommendations_df_category, user_id, category_filter_arn , category)
    
recommendations_df_category

,accessories,apparel,beauty,books,electronics,floral,footwear
0,Sleek Urban Explorer Backpack,Rustic Burlap Throw Blanket,Lumberjack's Cozy Comfort Set,Vibrant Mexican Artisan Bowls Set,Modern Living Room Entertainment Set,Autumn Harvest Wreath,Chic Crossover Sandals
1,Urban Graffiti Question Backpack,Classic Denim Button-Down Shirt,Luxury Artisan Soap Collection,French Cuisine Masterpiece Print,Immersive Waterfall Curved Monitor,Rustic Charm Floral Wreath,Blossom Waltz Floral Stilettos
2,Urban Stealth Backpack,Geometric Bliss Can Coolers,Vibrant Coral Dental Companion,Great Wall Adventure Package,Creative Workspace Multimedia Monitor,Vibrant Hyacinth Harmony Bouquet,Elegant Bow Slingback Flats
3,Caramel Delight Leather Tote,Mystic Veil Transformation Wrap,Tranquil Trio Bath Bombs,China Adventure Travel Poster,Modern Minimalist Home Office Setup,Enchanted Garden Centerpiece,Chic Comfort Slide Sandals
4,Alpine Explorer Backpack,Ocean Breeze Waterproof Jacket,Eco-Friendly Cotton Swab Set,Vibrant Mexico Travel Poster,Rustic Chic Smart TV Stand,Vibrant Urban Bouquet,Classic Elegance Stiletto Pumps
5,Eco-Friendly Canvas Tote Bag,Elegant Work-Ready Shirt,Sleek Stainless Steel Nail Clipper,Spain Wanderlust Poster,Gravity-Defying Camera Capture Kit,Festive Door Decor Duo,Checkered Charm Skate Sneakers
6,Adventure Pro Photography Backpack,Classic White Button-Down Shirt,Cozy Winter Sweater Collection,Indonesian Cuisine Cookbook,Mountain View Pro Camera Kit,Enchanted Garden Bridal Bouquet,Vibrant Leap Canvas Sneakers
7,Eco-Chic Kraft Paper Tote,Urban Explorer Stealth Backpack,Eco-Friendly Bamboo Toothbrush Set,Paris Panorama Travel Poster,Modern Zen Fireplace Entertainment Center,Elegant Garden Urn Arrangement,Elegance Stride Nude Heels
8,Rustic Charm Backpack,Rustic Charm Striped Sweater,Chic Lips Color Pop Set,Authentic Turkish Falafel Kit,Sleek Horizon Smart TV,Regal Radiance Bouquet,Rugged Terrain Work Boots
9,Urban Explorer Roll-Top Backpack,Urban Chic Oversized Blazer,Lush Lime Luxury Towel Set,Egyptian Pyramid Adventure Package,Pro Cinematographer's Dream Rig,Blushing Elegance Rose Bouquet,Eco-Bliss Massage Sandals


As an alternative to some of the steps above we could also have used the return item metadata functionality of personalize.

<div class="alert alert-block alert-warning">
<b>Important:</b> When you enable metadata in recommendations, you incur additional costs. For more information see Amazon Personalize pricing: https://aws.amazon.com/personalize/pricing/.
</div>

In [31]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 5,
    metadataColumns = {
            "ITEMS": ["PRODUCT_DESCRIPTION", "PRICE"]
        }
)

get_recommendations_response

{'ResponseMetadata': {'RequestId': 'c279fd25-c76d-4df9-b69b-ab7fa7060a63',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 21 Aug 2024 22:59:19 GMT',
   'content-type': 'application/json',
   'content-length': '2498',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c279fd25-c76d-4df9-b69b-ab7fa7060a63'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9',
   'metadata': {'price': '130.0',
    'product_description': 'Elevate your space with our sleek Minimalist Circular Wall Mirror. This modern, frameless design features a large round reflective surface that adds depth and light to any room. Perfect for entryways, bathrooms, or as a focal point in living areas.'}},
  {'itemId': '3999c48f-4ccf-463b-8e0b-f28dcdc469d1',
   'metadata': {'price': '13000.0',
    'product_description': 'Transform your living space into a concert hall with this stunning black grand piano. Featuring a sleek design, premium craftsmanship, and a matching adjustab

## Real-time Events<a class="anchor" id="event-tracking"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in retail workloads, because what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [32]:
event_tracker_name = 'personalize-poc-event-tracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd tracking_id = {}'.format(tracking_id))
    

{
  "eventTrackerArn": "arn:aws:personalize:us-east-1:809697808660:event-tracker/0614ea09",
  "trackingId": "f783dbba-788c-4f82-ba49-664889df3df6",
  "ResponseMetadata": {
    "RequestId": "72a23480-246b-4655-bdc6-f589971606da",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 21 Aug 2024 22:59:19 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "connection": "keep-alive",
      "x-amzn-requestid": "72a23480-246b-4655-bdc6-f589971606da",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}

Creating the event_tracker with event_tracker_arn = arn:aws:personalize:us-east-1:809697808660:event-tracker/0614ea09

And trackingId = f783dbba-788c-4f82-ba49-664889df3df6


### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [33]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

EventTracker: CREATE PENDING
EventTracker: ACTIVE


### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker. Since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. This challenge can be solved by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows applications to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, client applications can pass the provisional user ID to the user management system when creating a new user account. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Using the Purchased Products Filter

To use purchased products filter, we will request ranked recommendations for a random user. Then we will send a `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter. The purchased product would be excluded from the new recommendations.

In [34]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = 456
display(user_metadata_df[user_metadata_df['id']==user_id])

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
455,456,True,M,Austin,Jensen,austin.jensen@example.com,35,Austin Jensen,user456,housewares_floral_seasonal,all_discounts,{},{'ios': {'anonymous_id': '1212fae7-a8eb-4709-b...,"[{'first_name': 'Austin', 'last_name': 'Jensen..."


In [35]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Ethereal Blush Bouquet,floral,bouquet
1,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Precision Pro Red Screwdriver,tools,screwdriver
2,1e96e374-be23-4c97-b87e-b5c45cb8999f,Essential Italian Cookware Set,housewares,kitchen
3,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Lavender Bliss Artisan Soap Set,beauty,bathing
4,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Glow Jack-O'-Lantern Set,seasonal,halloween
5,01a8978b-2a84-4dbd-acc4-aff74a468681,Cheerful Succulent Planter Pot,homedecor,decorative
6,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Vibrant Clash Red Blazer Set,apparel,jacket
7,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Active Pup Frisbee Set,outdoors,pet
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Fresh-Cut Exotic Kiwi Duo,groceries,fruits
9,5d37a44b-d121-426e-b528-59e603ba5923,Egyptian Pyramid Adventure Package,books,travel


Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [36]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')
print(f'Product name: {get_item_name_from_id ( product_id_to_purchase)}')

Product to simulate purchasing: 4bb66b8a-cf13-4959-87ce-ca506fa568a2
Product name: Ethereal Blush Bouquet


Next, let's send a `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [37]:
response = personalize_events.put_events(
    trackingId = tracking_id,
    userId = str(user_id),
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "576a2815-eccf-4aaa-b6b2-add41f642eba",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 21 Aug 2024 22:59:34 GMT",
      "content-type": "application/json",
      "content-length": "0",
      "connection": "keep-alive",
      "x-amzn-requestid": "576a2815-eccf-4aaa-b6b2-add41f642eba"
    },
    "RetryAttempts": 0
  }
}


Finally, let's retrieve recommendations for the user again, but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.

In [38]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id),
    filterArn = purchase_filter_arn
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Precision Pro Red Screwdriver,tools,screwdriver
1,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Lavender Bliss Artisan Soap Set,beauty,bathing
2,1e96e374-be23-4c97-b87e-b5c45cb8999f,Essential Italian Cookware Set,housewares,kitchen
3,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Glow Jack-O'-Lantern Set,seasonal,halloween
4,01a8978b-2a84-4dbd-acc4-aff74a468681,Cheerful Succulent Planter Pot,homedecor,decorative
5,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Vibrant Clash Red Blazer Set,apparel,jacket
6,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Active Pup Frisbee Set,outdoors,pet
7,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Fresh-Cut Exotic Kiwi Duo,groceries,fruits
8,5d37a44b-d121-426e-b528-59e603ba5923,Egyptian Pyramid Adventure Package,books,travel
9,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Rustic Charm Sofa Set,furniture,sofas


The following code will raise an assertion error if the product we just purchased is still recommended.

In [39]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

Purchased item filtered from recommendations for user!


In [71]:
%store workshop_dataset_group_arn
%store region
%store role_name
%store s3_access_policy_arn

# Variables required for Optional Notebook 1 - Batch Inference
%store users
%store unranked_product_ids
%store bucket_name
%store workshop_rerank_solution_version_arn
%store item_metadata_df
%store category_filter_arn
%store tracking_id

Stored 'workshop_dataset_group_arn' (str)
Stored 'region' (str)
Stored 'role_name' (str)
Stored 's3_access_policy_arn' (str)
Stored 'users' (list)
Stored 'unranked_product_ids' (list)
Stored 'bucket_name' (str)
Stored 'workshop_rerank_solution_version_arn' (str)
Stored 'item_metadata_df' (DataFrame)
Stored 'category_filter_arn' (str)
Stored 'tracking_id' (str)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`Retail_04_Clean_Up.ipynb`](Retail_04_Clean_Up.ipynb).

You can choose to head to any of the optional notebooks in this folder to continue experimenting with Amazon Personalize.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: Run optional notebooks BEFORE you run the clean-up notebook.
</div>
